In [7]:
import requests
import json
import os
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup

In [8]:
class Weather_API:
    def __init__ (self, keyword):
        self.keyword = keyword
        
    def json_print(self, obj):
        # create a formatted string of the Python JSON object
        with open('api_data.txt', 'w') as json_file:
            json.dump(obj, json_file)
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)
        
    def create_dataframe(self, obj):
        
        # creating a dataframe from nested JSON objects
        FIELDS = ["source.id", "source.name", "author", "title", "description", "url", "urlToImage", "publishedAt", "content"]
        df = pd.json_normalize(obj['articles'])
        final_df = df[FIELDS]
        #final_df.set_index('source.id', inplace = True)
        display(final_df.head())

    
    def news_api(self):
        
        # Use the news-api to obtain articles published from
        url = ('https://newsapi.org/v2/everything?'
       'q={keyword}&'
       'apiKey=4e70cabb80884db08524a28ac33cdc1d'.format(keyword = self.keyword))
        
        
        response = requests.get(url)
        if (response.status_code == 200):
            print('API call successful!')
            json_response = response.json()
            if(len(json_response['articles']) == 0):
                print('No News Articles Found')
            else:
                
                # Print a String in Json Format
                self.json_print(json_response)
                
                # Create a pandas DataFrame
                self.create_dataframe(json_response)
                    
        else:
            print('Status code: ', response.status_code)

In [9]:
class Web_Scraping:
    
    def __init__(self, location):
        self.location = location
        
    def selenium_webdriver(self):
        
        # Start the Driver
        driver = webdriver.Chrome(executable_path = r"C:\Users\Swpnilkomatwar\Downloads\chromedriver_win32\chromedriver.exe")
        
        # Hit the url of NASA Earth Data website and wait for 15 seconds.
        url = ('https://earthdata.nasa.gov/search?q={location}'.format(location = self.location))
        driver.get(url)
        time.sleep(15)
        
        # Driver scrolls down 25 times to load the table.
        for i in range(0,30):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(10)
            
        # Fetch the webpage and store in a variable.
        webpage = driver.page_source
        
        # Parse the page using BeautifulSoup
        HTMLPage = BeautifulSoup(webpage, 'html.parser')
        
        titles = []
        description = []
        links = []

        for lists in HTMLPage.find_all(class_ = 'result'):
            if (lists.span.text != '' and len(lists.find_all('p')) != 0):
                titles.append(lists.span.text)
                description.append(lists.find('p', class_ = '').text)
                links.append(lists.find('p', class_ = 'search-link').text)
        
        # Create a DataFrame
        df = pd.DataFrame(list(zip(titles, description, links)),
               columns =['title', 'description', 'link'])
        
        display(df)
        
        # Store to csv file
        df.to_csv('ws.csv', sep=',', index=False,header=True)
        
        print('Web Scraping Successful!')

In [ ]:
keyword = input('Enter Keyword to be searched: ').lower()
w_api = Weather_API(keyword)
w_api.news_api()

location = input('Enter Location: ').lower()
ws = Web_Scraping('India')
ws.selenium_webdriver()

Enter Keyword to be searched: Pune
API call successful!
{
    "articles": [
        {
            "author": "ugc",
            "content": "In the heart of Pune,India, a bustling market teems with life in the Tulshibaugn area. Throngs of people visit the famous markets intricate crisscrossing lanes every day to shop for all kinds of ware\u2026 [+1215 chars]",
            "description": "In the heart of Pune, India, a bustling market teems with life in the Tulshibaugn area. Throngs of people visit the famous market's intricate crisscrossing lanes every day to shop for all kinds of wares and products, from food and clothing to artwork and reli\u2026",
            "publishedAt": "2024-02-09T16:00:00Z",
            "source": {
                "id": null,
                "name": "Atlas Obscura"
            },
            "title": "Tulshibaug Ram Temple in Pune, India",
            "url": "https://www.atlasobscura.com/places/tulshibaug-ram-temple-pune",
            "urlToImage": "https://img.

,source.id,source.name,author,title,description,url,urlToImage,publishedAt,content
0,None,Atlas Obscura,ugc,"Tulshibaug Ram Temple in Pune, India","In the heart of Pune, India, a bustling market...",https://www.atlasobscura.com/places/tulshibaug...,https://img.atlasobscura.com/5leSq65fQaTi_EM1m...,2024-02-09T16:00:00Z,"In the heart of Pune,India, a bustling market ..."
1,None,Atlas Obscura,ugc,"Lal Mahal in Pune, India",Pune shares a glorious relationship with histo...,https://www.atlasobscura.com/places/lal-mahal,https://img.atlasobscura.com/rvYszS9mznQn1dT6J...,2024-02-12T18:00:00Z,Pune shares a glorious relationship with histo...
2,None,The Indian Express,Express News Service,Classmate booked over suicide of MBA student i...,Police registered an FIR at the Hinjewadi poli...,https://indianexpress.com/article/cities/pune/...,https://images.indianexpress.com/2024/02/pune-...,2024-02-05T15:04:49Z,Four months after an MBA student allegedly die...
3,None,Atlas Obscura,ugc,"Shree Nageshwar Shiva Temple in Pune, India",The older parts of Pune have been divided into...,https://www.atlasobscura.com/places/shree-nage...,https://img.atlasobscura.com/vAkIIq5mkMRmL6qeT...,2024-01-30T15:00:00Z,The older parts of Pune have been divided into...
4,None,The Indian Express,Anuradha Mascarenhas,Indian Academy of Pediatrics launches ‘IAP Ki ...,The Indian Academy of Pediatrics has taken up ...,https://indianexpress.com/article/cities/pune/...,https://images.indianexpress.com/2024/02/Untit...,2024-02-13T16:49:13Z,With anaemia being a major health concern in t...
